In [8]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import botocore
import boto3
import time
from opensearchpy import RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from langchain.text_splitter import MarkdownHeaderTextSplitter
import os
import pandas as pd
from io import StringIO
import math
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains import RetrievalQA
from langchain.llms import Bedrock
from langchain.prompts import ChatPromptTemplate
from PyPDF2 import PdfReader
from IPython.display import Markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import Bedrock
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFDirectoryLoader
import json
from langchain_community.document_loaders import S3DirectoryLoader

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [9]:
client = boto3.client('opensearchserverless')
service = 'aoss'
region = 'us-east-1'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, service, session_token=credentials.token)

In [10]:
client = boto3.client('opensearchserverless', region_name=region)

In [11]:
def createEncryptionPolicy(client):
    try:
        response = client.create_security_policy(
            description='Encryption policy for RAG collections',
            name='rag-policy',
            policy="""
                {
                    \"Rules\":[
                        {
                            \"ResourceType\":\"collection\",
                            \"Resource\":[
                                \"collection\/rag-*\"
                            ]
                        }
                    ],
                    \"AWSOwnedKey\":true
                }
                """,
            type='encryption'
        )
        print('\nEncryption policy created:')
        print(response)
    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'ConflictException':
            print(
                '[ConflictException] The policy name or rules conflict with an existing policy.')
        else:
            raise error


def createNetworkPolicy(client):
    try:
        response = client.create_security_policy(
            description='Network policy for RAG collections',
            name='rag-policy',
            policy="""
                [{
                    \"Description\":\"Public access for rag collection\",
                    \"Rules\":[
                        {
                            \"ResourceType\":\"dashboard\",
                            \"Resource\":[\"collection\/rag-*\"]
                        },
                        {
                            \"ResourceType\":\"collection\",
                            \"Resource\":[\"collection\/rag-*\"]
                        }
                    ],
                    \"AllowFromPublic\":true
                }]
                """,
            type='network'
        )
        print('\nNetwork policy created:')
        print(response)
    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'ConflictException':
            print(
                '[ConflictException] A network policy with this name already exists.')
        else:
            raise error


def createAccessPolicy(client):
    try:
        response = client.create_access_policy(
            description='Data access policy for RAG collections',
            name='rag-policy',
            policy="""
                [{
                    \"Rules\":[
                        {
                            \"Resource\":[
                                \"index\/rag-*\/*\"
                            ],
                            \"Permission\":[
                                \"aoss:CreateIndex\",
                                \"aoss:DeleteIndex\",
                                \"aoss:UpdateIndex\",
                                \"aoss:DescribeIndex\",
                                \"aoss:ReadDocument\",
                                \"aoss:WriteDocument\"
                            ],
                            \"ResourceType\": \"index\"
                        },
                        {
                            \"Resource\":[
                                \"collection\/rag-*\"
                            ],
                            \"Permission\":[
                                \"aoss:CreateCollectionItems\"
                            ],
                            \"ResourceType\": \"collection\"
                        }
                    ],
                    \"Principal\":[
                        \"arn:aws:iam::646932767172:role/RAGrol"
                    ]
                }]
                """,
            type='data'
        )
        print('\nAccess policy created:')
        print(response)
    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'ConflictException':
            print(
                '[ConflictException] An access policy with this name already exists.')
        else:
            raise error


def createCollection(client):
    """Creates a collection"""
    try:
        response = client.create_collection(
            name='rag-maintenance-collection',
            type='VECTORSEARCH'
        )
        return(response)
    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'ConflictException':
            print(
                '[ConflictException] A collection with this name already exists. Try another name.')
        else:
            raise error


def waitForCollectionCreation(client):
    """Waits for the collection to become active"""
    response = client.batch_get_collection(
        names=['rag-maintenance-collection'])
    # Periodically check collection status
    while (response['collectionDetails'][0]['status']) == 'CREATING':
        print('Creating collection...')
        time.sleep(30)
        response = client.batch_get_collection(
            names=['rag-maintenance-collection'])
    print('\nCollection successfully created:')
    print(response["collectionDetails"])
    host = (response['collectionDetails'][0]['collectionEndpoint'])
    final_host = host.replace("https://", "")
    print('host', final_host)


def main():
    createEncryptionPolicy(client)
    createNetworkPolicy(client)
    createAccessPolicy(client)
    createCollection(client)
    waitForCollectionCreation(client)


if __name__ == "__main__":
    main()

[ConflictException] The policy name or rules conflict with an existing policy.
[ConflictException] A network policy with this name already exists.
[ConflictException] An access policy with this name already exists.
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...

Collection successfully created:
[{'arn': 'arn:aws:aoss:us-east-1:646932767172:collection/zycg86yzxkvuq5dthaa8', 'collectionEndpoint': 'https://zycg86yzxkvuq5dthaa8.us-east-1.aoss.amazonaws.com', 'createdDate': 1709126258779, 'dashboardEndpoint': 'https://zycg86yzxkvuq5dthaa8.us-east-1.aoss.amazonaws.com/_dashboards', 'id': 'zycg86yzxkvuq5dthaa8', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1709126617971, 'name': 'rag-maintenance-collection', 'standbyReplicas': 'ENABLED', 'status': 'ACTIVE', 'type': 'VECTO

In [95]:
def split_text_pdf(data):
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap  = 150)
    text_chunks = text_splitter.create_documents([data])
    return text_chunks

In [122]:
s3 = boto3.client('s3')

bucket_name = 'maintenance-documentation-rag'

def process_pd_documents(bucket_name):
    texts = []
    objects = s3.list_objects_v2(Bucket=bucket_name)['Contents']
    for obj in objects:
        key = obj['Key']
        if key.endswith(".pdf"):
            title = key[0:-4]
            pdf_response = s3_client.get_object(Bucket=bucket_name, Key=pdf_file)
            file_content = pdf_response['Body'].read()
            pdf_text = ""
            with pdfplumber.open(io.BytesIO(file_content)) as pdf:
                for page in pdf.pages:
                    pdf_text += page.extract_text()
                    pdf_text = pdf_text.replace(".","")
                    chunks = split_text_pdf(pdf_text)
                    for chunk_index, chunk in enumerate(chunks):
                        entry = {'title': title, 'text': chunk.page_content}
                        texts.append(entry)
    df = pd.DataFrame(texts)
    return df

In [124]:
df = process_pd_documents(bucket_name)

In [125]:
def clean_text(df):
    df = df.fillna('.')
    df = df.replace('\n',' ', regex=True)
    df = df.replace('#',' ', regex=True)
    df = df.replace('-',' ', regex=True)
    df = df.replace('__',' ', regex=True)
    df = df.replace('@',' ', regex=True)
    df = df.replace('/',' ', regex=True)
    df = df.replace('`',' ', regex=True)
    df = df.replace(' +',' ', regex=True)
    df = df.replace('>',' ', regex=True)
    df = df.replace('<',' ', regex=True)
    df = df.replace('{',' ', regex=True)
    df = df.replace('}',' ', regex=True)
    df = df.replace('!',' ', regex=True)
    df = df.replace('[^\w\s]','', regex=True)
    return df

In [126]:
df = clean_text(df)

In [127]:
df.head()

,title,text
0,1 NORMA IEC 60300 3 11 RCM Guia de Aplicación ...,INTERNATIONAL ISO STANDARD 17359 Third edition...
1,1 NORMA IEC 60300 3 11 RCM Guia de Aplicación ...,INTERNATIONAL ISO STANDARD 17359 Third edition...
2,1 NORMA IEC 60300 3 11 RCM Guia de Aplicación ...,INTERNATIONAL ISO STANDARD 17359 Third edition...
3,1 NORMA IEC 60300 3 11 RCM Guia de Aplicación ...,61 Identification of equipment 3 62 Identifica...
4,1 NORMA IEC 60300 3 11 RCM Guia de Aplicación ...,INTERNATIONAL ISO STANDARD 17359 Third edition...


In [131]:
loader = DataFrameLoader(df, page_content_column="text")
docs = loader.load()
for i in loader.lazy_load():
    print(i)
    break
for doc in docs:
    doc.metadata = {k: (v if not (isinstance(v, float) and math.isnan(v)) else "null") for k, v in doc.metadata.items()}

page_content='INTERNATIONAL ISO STANDARD 17359 Third edition 2018 01 Condition monitoring and diagnostics of machines  General guidelines Surveillance et diagnostic détat des machines  Lignes directrices générales Reference number ISO 173592018E  ISO 2018 Provided by IHS Markit under license with ANSI' metadata={'title': '1 NORMA IEC 60300 3 11 RCM Guia de Aplicación Mantenimiento Centrado en la Fiabilidad'}


In [132]:
import boto3
from langchain.embeddings import BedrockEmbeddings

bedrock_client = boto3.client(service_name='bedrock-runtime', 
                              region_name='us-east-1')
embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_client)

In [133]:
aoss_endpoint = os.environ.get("AOSS_ENDPOINT", "default_endpoint")

In [134]:
region = 'us-east-1'
service = 'aoss'
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)
aoss_client = OpenSearch(
    hosts=[{'host': aoss_endpoint, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

In [135]:
index_name = 'rag-maintenance-doc'

In [136]:
documents = [docs]

In [ ]:
%%time

for docs in documents:
    oss = OpenSearchVectorSearch.from_documents(
        docs,
        embeddings,
        opensearch_url=f'https://{aoss_endpoint}:443',
        http_auth=auth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        index_name=index_name,
        bulk_size=20000,
        timeout=400,
    )
    print(".", end="")

.CPU times: user 57.9 s, sys: 3.07 s, total: 1min
Wall time: 33min 7s


In [140]:
docsearch = OpenSearchVectorSearch(
    index_name=index_name,
    embedding_function=embeddings,
    opensearch_url=f'https://{aoss_endpoint}:443',
    http_auth=auth,
    is_aoss=True,
    timeout = 60,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection)

In [50]:
retriever = docsearch.as_retriever(search_kwargs={"k": 5})

In [51]:
query = "Qué es el PEGA ?"

In [52]:
documents_retrieved = docsearch.similarity_search_with_score(query, k=5)
documents_retrieved

[(Document(page_content='El PEGA es una pieza clave de información documentada que se utiliza para traducir los objetivos de la organización en objetivos de gestión de activos y proporcionar el marco para la planificación, la priorización y la toma de decisiones para la implementación de todas las demás actividades de gestión de activos para garantizar la alineación. Alinear los objetivos de gestión de activos con los objetivos de la organización, así como vincular los planes de portafolio de activos con los planes financieros, puede mejorar la efectividad y la eficiencia de la organización e informar las necesidades de financiamiento a largo plazo de la organización.\n\nLos requisitos para el PEGA están distribuidos en estas diversas subcláusulas de ISO 55001: 2014, 4.1, 4.3, 4.4, 5.1, 5.3, 6.2.1 y 6.2.2.\n\nEl PEGA puede tener un período de tiempo suficientemente largo para abordar la vida útil completa de los activos que podría extenderse más allá del propio período de tiempo de pla